# Evaluate speaker verification with PCA

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.chdir('../..')
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

In [13]:
import torch
from sklearn.decomposition import PCA
import numpy as np

from notebooks.notebooks_utils import (
    load_models,
    evaluate_models,
    create_metrics_df
)

from sslsv.evaluation.CosineSVEvaluation import CosineSVEvaluation, CosineSVEvaluationTaskConfig

In [17]:
models = load_models([
    './configs/simclr_ecapa.yml',
])

In [36]:
class CosineSVWithPCAEvaluation(CosineSVEvaluation):
    
    def _prepare_evaluation(self, trials):
        super()._prepare_evaluation(trials)
        
        # Train PCA
        Z = np.array([t.numpy().squeeze(0) for t in self.test_embeddings.values()])
        self.pca = PCA(n_components=200)
        self.pca.fit(Z)
    
    def _get_sv_score(self, a, b):
        enrol = self.test_embeddings[a]
        test = self.test_embeddings[b]
        
        enrol = torch.from_numpy(self.pca.transform(enrol.numpy()))
        test = torch.from_numpy(self.pca.transform(test.numpy()))

        score = self._compute_score(enrol, test)

        return score.item()

In [18]:
evaluate_models(models, CosineSVEvaluation, CosineSVEvaluationTaskConfig())
create_metrics_df(models)

Extracting test embeddings: 100%|█████████████████████████████████████████████████████████| 74/74 [00:14<00:00,  5.14it/s]


,voxceleb1_test_O/eer,voxceleb1_test_O/mindcf
Model,,
simclr_ecapa,6.8557,0.6059


In [37]:
evaluate_models(models, CosineSVWithPCAEvaluation, CosineSVEvaluationTaskConfig())
create_metrics_df(models)

Extracting test embeddings: 100%|█████████████████████████████████████████████████████████| 74/74 [00:14<00:00,  5.15it/s]


,voxceleb1_test_O/eer,voxceleb1_test_O/mindcf
Model,,
simclr_ecapa,6.0153,0.6194
